## First Neural Network: Image Classification 

Objectives:
- Train a minimal image classifier on [MNIST](https://paperswithcode.com/dataset/mnist) using PyTorch
- Usese PyTorch and torchvision

In [1]:
# The usual imports

import torch  # 导入PyTorch库
import torch.nn as nn  # 导入PyTorch的神经网络模块
import torchvision  # 导入PyTorch的计算机视觉库
import torchvision.transforms as transforms  # 导入PyTorch的图像转换模块

In [2]:
# load the data

# 定义了一个名为 ReshapeTransform 的类
class ReshapeTransform:
    def __init__(self, new_size):   # 构造函数 __init__，用于初始化新的大小 new_size
        self.new_size = new_size

    def __call__(self, img):            # 定义了 __call__ 方法，该方法接收一个图像 img，并使用 torch.reshape 函数将图像重塑为指定的新大小 new_size，然后返回重塑后的图像
        return torch.reshape(img, self.new_size)

# 创建了一个变换（transformation）的组合 transformations，使用了 transforms.Compose 函数
transformations = transforms.Compose([
                                transforms.ToTensor(),                          # 将图像转换为张量形式
                                transforms.ConvertImageDtype(torch.float32),    # 将图像数据类型转换为 torch.float32
                                ReshapeTransform((-1,))                         # 使用自定义的 ReshapeTransform 类将图像重塑为一维形状
                                ])

# 创建了训练集 trainset 和测试集 testset，使用了 MNIST 数据集
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transformations)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transformations)

In [3]:
# check shape of data

trainset.data.shape, testset.data.shape     # 获取了 trainset 和 testset 数据集的形状（shape）

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [4]:
# data loader

BATCH_SIZE = 128

# 创建了一个训练数据加载器 train_dataloader，使用了 torch.utils.data.DataLoader 类
train_dataloader = torch.utils.data.DataLoader(trainset,                # 训练数据集
                                               batch_size=BATCH_SIZE,   # 每个批次的样本数量
                                               shuffle=True,            # 在每个时期（epoch）开始前打乱数据集的顺序，以增加样本之间的独立性
                                               num_workers=0)           # 在数据加载过程中使用的子进程数量，这里设定为 0 表示不使用额外的子进程

test_dataloader = torch.utils.data.DataLoader(testset, 
                                              batch_size=BATCH_SIZE,
                                              shuffle=False, 
                                              num_workers=0)

In [5]:
# model

# 定义了一个神经网络模型 model，使用了 nn.Sequential 类
# nn.Linear 是线性层（fully connected layer），用于定义模型的线性变换
# 包含了两个线性层和一个ReLU激活函数。输入层的大小为 784，输出层的大小为 10，中间的隐藏层大小为 512
# nn.Sequential 类是一个顺序容器，它按照给定的顺序组合各个网络层，将输入数据依次传递给这些层进行计算
model = nn.Sequential(nn.Linear(784, 512), nn.ReLU(), nn.Linear(512, 10))

In [6]:
# training preparation

trainer = torch.optim.RMSprop(model.parameters())   # 优化器
loss = nn.CrossEntropyLoss()                        # 损失函数

In [7]:
def get_accuracy(output, target, batch_size):       # 定义了一个函数 get_accuracy，用于计算准确率。该函数接收三个参数：output 是模型的输出结果，target 是真实的目标标签，batch_size 是批次的大小
    # Obtain accuracy for training round
    # 通过 torch.max(output, 1)[1] 获取每个样本预测结果的类别索引，然后使用 .view(target.size()).data == target.data 进行比较，得到一个布尔值的张量，表示预测是否正确。接着使用 .sum() 计算预测正确的数量，赋值给变量 corrects
    corrects = (torch.max(output, 1)[1].view(target.size()).data == target.data).sum()
    # 通过计算正确数量与批次大小的比例，并乘以 100，得到准确率
    accuracy = 100.0 * corrects/batch_size
    # accuracy.item() 将准确率转换为标量值，并返回该值
    return accuracy.item()

In [8]:
# train

for ITER in range(5):       # 迭代
    train_acc = 0.0
    train_running_loss = 0.0

    # 将训练模式设置为 model.train()，然后遍历训练数据加载器 train_dataloader，获取每个批次的输入 X 和目标标签 y
    model.train()
    for i, (X, y) in enumerate(train_dataloader):
        # 使用模型 model 进行前向传播，得到预测结果 output。然后使用定义的损失函数 loss 计算损失值 l
        output = model(X)
        l = loss(output, y)

        # update the parameters
        # 通过 l.backward() 计算损失的梯度，并使用优化器 trainer.step() 更新模型的参数。然后使用 trainer.zero_grad() 清零优化器的梯度
        l.backward()
        trainer.step()
        trainer.zero_grad()

        # gather metrics
        # 通过调用 get_accuracy(output, y, BATCH_SIZE) 计算当前批次的准确率，并累加到 train_acc 中。同时，将当前批次的损失值 l.detach().item() 累加到 train_running_loss 中
        train_acc += get_accuracy(output, y, BATCH_SIZE)
        train_running_loss += l.detach().item()
    # 输出当前 epoch 的训练损失和训练准确率
    print('Epoch: %d | Train loss: %.4f | Train Accuracy: %.4f' \
          %(ITER+1, train_running_loss / (i+1),train_acc/(i+1)))

Epoch: 1 | Train loss: 0.9943 | Train Accuracy: 91.7344
Epoch: 2 | Train loss: 0.1334 | Train Accuracy: 95.9422
Epoch: 3 | Train loss: 0.1030 | Train Accuracy: 96.8767
Epoch: 4 | Train loss: 0.0845 | Train Accuracy: 97.4997
Epoch: 5 | Train loss: 0.0735 | Train Accuracy: 97.8811


### Other things to try

- Evaluate on test set
- Plot loss curve
- Add more layers to the model